In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
import googleapiclient.discovery
import pandas as pd

# Video Ids

In [24]:
def get_video_ids(youtube_service_object, channel_id: str, published_after, published_before, search_term: str = None):
    """Connects to the YouTube Data API using 'search' and returns video Ids for a specified request.

    
    Parameters
    ----------
    service_object : googleapiclient object
        a service object created using `googleapiclinet.discovery.build`

    channel_Id : str
        The id of the YouTube channel you want to search for videos.

    published_after : datetime
        an RFC 3339 formatted date-time value (1970-01-01T00:00:00Z).

    published_before : datetime
        an RFC 3339 formatted date-time value (1970-01-01T00:00:00Z).

    query : str
        A search term if you wish to narrow down the search using keywords. See Notes for 
        further information.



    Returns
    --------
    video_ids : list
        A list of the videoId values obtained from the request.

    
    Notes
    ------
    query : str
        Your request can also use the Boolean NOT (-) and OR (|) operators to exclude videos or 
        to find videos that are associated with one of several search terms. For example, to 
        search for videos matching either "boating" or "sailing", set the q parameter value to 
        boating|sailing. Similarly, to search for videos matching either "boating" or "sailing" 
        but not "fishing", set the q parameter value to boating|sailing -fishing. Note that the 
        pipe character must be URL-escaped when it is sent in your API request. The URL-escaped 
        value for the pipe character is %7C.


    References
    -----------
        https://developers.google.com/youtube/v3/docs/search
    
    """
    
    # make a request to the youtube api
    request = youtube_service_object.search().list(
        channelId=channel_id,
        publishedAfter=published_after,
        publishedBefore=published_before,
        q=search_term,
        part="snippet", 
        type="video",
        order="date",
        maxResults=50,
    )
    response = request.execute()
    
    video_ids = []

    # loop through response and store video Ids in a list
    for i, v in enumerate(range(len(response["items"]))):
        video_ids.append(response["items"][i]["id"]["videoId"])
    
    next_page_token = response.get("nextPageToken", None)
    more_pages = True
    
    while more_pages == True:
        if next_page_token is None:
            more_pages = False
    
        else: 
            # make a request to the youtube api to get the next page results
            request = youtube_service_object.search().list(
            channelId=channel_id,
            publishedAfter=published_after,
            publishedBefore=published_before,
            q=search_term,
            part="snippet", 
            type="video",
            order="date",
            maxResults=50,
                pageToken=next_page_token
            )
            response = request.execute()
    
            for i, v in enumerate(range(len(response["items"]))):
                video_ids.append(response["items"][i]["id"]["videoId"])
            
            next_page_token = response.get("nextPageToken", None)
    
    return video_ids

# Test function

In [25]:
# create a youtube service object
api_service_name = "youtube"
api_version = "v3"
API_KEY = os.environ.get("API_KEY")

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=API_KEY)

In [34]:
# set channel and search details
bandai_namco_america_id = "UC_ntXHv-XdKCD7CPynVvnQw"
published_after = "2023-06-01T00:00:00Z"
published_before = "2023-12-20T00:00:00Z"
search_term = "tekken"

In [35]:
video_ids = get_video_ids(youtube_service_object=youtube, 
                          channel_id=bandai_namco_america_id,
                         published_after=published_after,
                         published_before=published_before,
                         search_term=search_term)

In [36]:
video_ids

['9jJiNa4HoD0',
 'EMZkmjE8wdw',
 'ToKJfywbe1o',
 '9D5vq-zq9y4',
 'y8JGUIF2pu4',
 'oeFfzCWif-Q',
 'bSCANspTDeE',
 'ucesGynb2Yk',
 'UcBcNOSoFzI',
 '8DVlK_QrZ-A',
 'Zc-yMi05vBA',
 'e1N4juHVqNc',
 'QH6s_o3dIic',
 '7skTtnpSb58',
 'bjzYbEjE-C4',
 'weVrUBszFIM',
 'Gw5nQaSF0CI',
 '3pGxqOFmIN4',
 'w0IqzD-gUOI',
 'PsCpewoF2E4',
 'cHnxJplTQuY',
 'qbUnCiTMCGE',
 'cIDK50IaVpg',
 'rDxrpSqYHD8',
 'YFJfLsJtVzM',
 'e7mqbmNb6eA',
 '_q26pgYDOV4',
 'n27RxZ7vnAU',
 'lDgv7CMIoRo',
 'nA-QTZbm_hU',
 'KHSwxMDibvE',
 'sNHv1y46dhs',
 'yowAmloydVY',
 '6hxZCQtpJ9w',
 '-l3AY19cn0M',
 'MxGp3wHXtNE',
 '4UkK_psUEVM',
 'maawo7O9Sg8',
 'Qkoba4YhbGo',
 'N5ZlZSnNyo0',
 'hEQTFXMQU7I',
 'flXHAFNT4sU',
 'RFO9Z_0wrKE',
 'kpmOhBWlDfc',
 '0Tk5YA4WRrg',
 'bMbDVh_OKZg',
 'UPPKjJgQT4A',
 'iILoqJlCa2s',
 'ODforBeu7_c',
 'DcDjyestr4Y',
 'Vs2piSWfofQ',
 '9Xgj0KYypVw',
 'LYdPz_GD8OQ',
 '6JLOD_a7or4',
 'LpxwBAxQ9Ug',
 'Nh5udl01YXc']

In [37]:
len(video_ids)

56

# Channel data

In [37]:
# make a request to the youtube api
request_channel_data = youtube.channels().list(
    part = "snippet,statistics,contentDetails", 
    # forUsername = "BandaiNamcoAmerica",
    id = "UC_ntXHv-XdKCD7CPynVvnQw"
)

### Explore response

In [38]:
response = request_channel_data.execute()
response

{'kind': 'youtube#channelListResponse',
 'etag': 'QeuQnxg-joWDMIF3K2-6bV0Ql1E',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': '38tdTXPvMYugZ3JFxaaUmMrhR_g',
   'id': 'UC_ntXHv-XdKCD7CPynVvnQw',
   'snippet': {'title': 'Bandai Namco Entertainment America',
    'description': 'Fun for all into the future! Tune in right here for new trailers and reveals for your favorite Bandai Namco games!\n\n\n',
    'customUrl': '@bandainamcoamerica',
    'publishedAt': '2006-09-19T17:48:06Z',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/-K3RYA88iYssAO0kKa5DKazS_6mrkBIbgpnHpykrGeROqEIZ18QA7JmD6KPiaXwL_JXNp3hf=s88-c-k-c0x00ffffff-no-rj',
      'width': 88,
      'height': 88},
     'medium': {'url': 'https://yt3.ggpht.com/-K3RYA88iYssAO0kKa5DKazS_6mrkBIbgpnHpykrGeROqEIZ18QA7JmD6KPiaXwL_JXNp3hf=s240-c-k-c0x00ffffff-no-rj',
      'width': 240,
      'height': 240},
     'high': {'url': 'https://yt3.ggpht.com/-K3RYA88iYssAO0kKa5DKaz

In [35]:
response['items'][0]['id']

'UC_ntXHv-XdKCD7CPynVvnQw'

In [36]:
response['items'][0]['statistics']['subscriberCount']

'1090000'

# Create dataframe from search response

In [38]:
# make a request to the youtube api
request = youtube.search().list(
    channelId=bandai_namco_america_id,
    publishedAfter=published_after,
    publishedBefore=published_before,
    q=search_term,
    part="snippet", 
    type="video",
    order="date",
    maxResults=50,
)
response = request.execute()

In [39]:
response["items"][0]["id"]["videoId"]

'9jJiNa4HoD0'

In [41]:
for i, v in enumerate(response["items"]):

    title = response["items"][i]["snippet"]["title"].lower()
    video_id = response["items"][i]["snippet"]["title"]

    if search_term in title:
        print(title)

tekken 8 – ultimate edition trailer
tekken 8 – official story trailer
tekken 8 exclusive story demo showcase
tekken 8 — yoshimitsu reveal &amp; gameplay trailer
tekken 8 — dragunov reveal &amp; gameplay trailer
tekken 8 – steve fox reveal &amp; gameplay trailer
tekken 8 — leo reveal &amp; gameplay trailer
tekken 8 — reina reveal &amp; gameplay trailer
tekken 8 – victor chevalier reveal &amp; gameplay trailer
tekken 8 – the return of legends - new characters reveal trailer
tekken 8 - premium collector&#39;s edition overview
tekken 8 - introduction feng v. asuka
tekken 8 - feng and closed beta test reveal trailer
tekken 8 – release date and exclusive content reveal trailer
tekken 8 — raven reveal &amp; gameplay trailer
tekken 8 — azucena reveal &amp; gameplay trailer
tekken talk live @ evo 2023!


In [37]:
len(response_tekken["items"])

30

- Use _search_ to get the video IDs
- Use the video IDs as input for _videos_ to get the video data
- Use video IDs as input for _commentThreads_ to get the comments for each video